# ReLLM

>[ReLLM](https://github.com/r2d4/rellm) 是一个包装本地 Hugging Face pipeline 模型用于结构化解码的库。
>它通过逐个生成 token 来工作。在每一步中，它会屏蔽不符合提供的部分正则表达式的 token。


**警告 - 这个模块仍然是实验性的**

In [1]:
!pip install rellm > /dev/null

### Hugging Face 基线

首先，让我们通过检查未经过结构化解码的模型的输出来确定定性基线。

In [2]:
import logging
logging.basicConfig(level=logging.ERROR)
prompt = """Human: "What's the capital of the United States?"
AI Assistant:{
  "action": "Final Answer",
  "action_input": "The capital of the United States is Washington D.C."
}
Human: "What's the capital of Pennsylvania?"
AI Assistant:{
  "action": "Final Answer",
  "action_input": "The capital of Pennsylvania is Harrisburg."
}
Human: "What 2 + 5?"
AI Assistant:{
  "action": "Final Answer",
  "action_input": "2 + 5 = 7."
}
Human: 'What's the capital of Maryland?'
AI Assistant:"""

In [3]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

hf_model = pipeline("text-generation", model="cerebras/Cerebras-GPT-590M", max_new_tokens=200)

original_model = HuggingFacePipeline(pipeline=hf_model)

generated = original_model.generate([prompt], stop=["Human:"])
print(generated)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generations=[[Generation(text=' "What\'s the capital of Maryland?"\n', generation_info=None)]] llm_output=None


***这样还不够印象深刻吗？它既没有回答问题，也没有遵循 JSON 格式！让我们尝试使用结构化解码器。***

## RELLM LLM Wrapper

让我们再试一次，这次提供一个匹配 JSON 结构格式的正则表达式。

In [4]:
import regex # Note this is the regex library NOT python's re stdlib module

# We'll choose a regex that matches to a structured json string that looks like:
# {
#  "action": "Final Answer",
# "action_input": string or dict
# }
pattern = regex.compile(r'\{\s*"action":\s*"Final Answer",\s*"action_input":\s*(\{.*\}|"[^"]*")\s*\}\nHuman:')

In [5]:
from langchain.experimental.llms import RELLM

model = RELLM(pipeline=hf_model, regex=pattern, max_new_tokens=200)

generated = model.predict(prompt, stop=["Human:"])
print(generated)

{"action": "Final Answer",
  "action_input": "The capital of Maryland is Baltimore."
}



**嘿！自由解析错误了。**